## Notebook for spike sorting from SGL data using Kilosort
Uses:
    - intan2kwik (https://github.com/zekearneodo/intan2kwik/blob/master/README.md)
    - mountainlab suite(https://github.com/flatironinstitute/mountainlab-js)
    - mountainsort https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md
    - mountainsort examples https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md

In [1]:
import socket
import os
import glob
import json
import shutil 
from typing import Union
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import subprocess
from datetime import timedelta
from importlib import reload
import shutil

# pipeline imports
from pipefinch.h5tools.kwik import kutil
from pipefinch.pipeline import probes
from pipefinch.pipeline import sglxutil as sglu
from pipefinch.neural.sort import kilo

from pipefinch.pipeline import filestructure as et

import logging

# Setup the logger
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
ch.setFormatter(formatter)
logger.addHandler(ch)
        
logger.info('Logger set')
logger.info('Hostname {}'.format(socket.gethostname()))

2019-08-01 19:35:29,524 - root - INFO - Logger set
2019-08-01 19:35:29,525 - root - INFO - Hostname zpikezorter


### Session parameters and raw files

In [2]:
reload(et)
bird = 'g4r4'
all_bird_sess = et.list_sessions(bird)
all_bird_sess

['20190711_01',
 '20190711_02',
 '20190711_03_tipref',
 '20190711_04',
 '20190712_01',
 '20190712_02',
 '20190712_03',
 '20190712_04',
 '20190712_05',
 '20190711_4800_01_g0',
 '20190713_01',
 '20190723_02',
 '20190712_01_extref_g0',
 '20190714_05',
 '20190714_06',
 '20190715_01',
 '20190715_02',
 '20190716_01',
 '20190716_02',
 '20190717_01',
 '20190717_02',
 '20190718_01',
 '20190718_02',
 '20190719_01',
 '20190719_02',
 '20190720_01',
 '20190721_01',
 '20190722_01',
 '20190722_02',
 '20190722_03',
 '20190722_04',
 '20190723_01',
 '20190723_03',
 '20190724_01',
 '20190713_02',
 '20190713_03',
 '20190713_04',
 '20190724_02',
 '20190714_01',
 '20190714_03',
 '20190714_04',
 '20190724_03',
 '20190725_01',
 '20190725_02',
 '20190725_03',
 '20190726_01',
 '20190726_02',
 '20190711_04_extref',
 '20190714_02']

In [29]:
reload(et)

sess_par = {'bird': bird,
           'sess': '20190711_03_tipref',
           'probe': 'probe_0', # probe to sort ('probe_0', 'probe_1') (to lookup in the rig_par which port to extract)
           'sort': 1, 
           'epoch': None, # for the subfolder in the neuropix data}
           }

exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], sess_par['sort'])

sort_params = {'adjacency_radius': -1,
              'detect_threshold': 2,
              'freq_min': 600}

ds_params = {'detect_sign': -1}

ks_params = {'use_gpu': 1,
            'auto_merge': 1,
            'filt_per_chan': 4,
            }

# visualization default parameters
viz_par = { 'evt_name': 'motif',
           'evt_signal': 'trig_perceptron',
            'evt_edge': 1,
            'pre_ms': -500,
            'post_ms': 300,
            
            'pre_samples': 0,
            'post_samples': 0,
            'span': 0,
            }

# convenient paths
kwik_folder = exp_struct['folders']['kwik']
ksort_folder = exp_struct['folders']['ksort']
raw_folder = exp_struct['folders']['raw']

In [30]:
exp_struct

{'folders': {'raw': '/mnt/microdrive/birds/g4r4/Ephys/raw/20190711_03_tipref',
  'kwik': '/data/experiment/microdrive/g4r4/Ephys/kwik/20190711_03_tipref',
  'msort': '/data/experiment/microdrive/g4r4/Ephys/msort/20190711_03_tipref',
  'ksort': '/data/experiment/microdrive/g4r4/Ephys/ksort/20190711_03_tipref'},
 'files': {'par': '/data/experiment/microdrive/g4r4/Ephys/ksort/20190711_03_tipref/params.json',
  'set': '/mnt/microdrive/birds/g4r4/Ephys/raw/20190711_03_tipref/settings.isf',
  'rig': '/mnt/microdrive/birds/g4r4/Ephys/raw/20190711_03_tipref/rig.json',
  'kwd': '/data/experiment/microdrive/g4r4/Ephys/kwik/20190711_03_tipref/streams.kwd',
  'kwik': '/data/experiment/microdrive/g4r4/Ephys/kwik/20190711_03_tipref/sort_1/spikes.kwik',
  'kwe': '/data/experiment/microdrive/g4r4/Ephys/kwik/20190711_03_tipref/events.kwe',
  'mda_raw': '/data/experiment/microdrive/g4r4/Ephys/msort/20190711_03_tipref/raw.mda',
  'bin_raw': '/data/experiment/microdrive/g4r4/Ephys/ksort/20190711_03_tipref

### Pick an epoch

In [31]:
sglu.list_sgl_epochs(sess_par)

['20190711_03_tipref_g0', '20190711_03_tipref_g1']

In [32]:
epoch = sglu.list_sgl_epochs(sess_par)[1]
sess_par['epoch'] = epoch
exp_struct = sglu.sgl_struct(sess_par, epoch)
exp_struct

2019-08-02 15:52:48,904 - pipefinch.pipeline.sglxutil - INFO - {'kwd': '/data/experiment/microdrive/g4r4/Ephys/kwik/20190711_03_tipref/20190711_03_tipref_g1/streams.kwd', 'kwe': '/data/experiment/microdrive/g4r4/Ephys/kwik/20190711_03_tipref/20190711_03_tipref_g1/events.kwe', 'mda_raw': '/data/experiment/microdrive/g4r4/Ephys/msort/20190711_03_tipref/20190711_03_tipref_g1/raw.mda'}


{'folders': {'raw': '/mnt/microdrive/birds/g4r4/Ephys/raw/20190711_03_tipref/20190711_03_tipref_g1',
  'kwik': '/data/experiment/microdrive/g4r4/Ephys/kwik/20190711_03_tipref/20190711_03_tipref_g1',
  'msort': '/data/experiment/microdrive/g4r4/Ephys/msort/20190711_03_tipref/20190711_03_tipref_g1',
  'ksort': '/data/experiment/microdrive/g4r4/Ephys/ksort/20190711_03_tipref/20190711_03_tipref_g1'},
 'files': {'par': '/data/experiment/microdrive/g4r4/Ephys/ksort/20190711_03_tipref/params.json',
  'set': '/mnt/microdrive/birds/g4r4/Ephys/raw/20190711_03_tipref/settings.isf',
  'rig': '/mnt/microdrive/birds/g4r4/Ephys/raw/20190711_03_tipref/rig.json',
  'kwd': '/data/experiment/microdrive/g4r4/Ephys/kwik/20190711_03_tipref/20190711_03_tipref_g1/streams.kwd',
  'kwik': '/data/experiment/microdrive/g4r4/Ephys/kwik/20190711_03_tipref/sort_1/spikes.kwik',
  'kwe': '/data/experiment/microdrive/g4r4/Ephys/kwik/20190711_03_tipref/20190711_03_tipref_g1/events.kwe',
  'mda_raw': '/data/experiment/mi

In [5]:
# in zpike
sess_folder = exp_struct['folders']['raw']
#in lookfar
#sess_folder = '/Users/zeke/experiment/stimsim/Ephys/2019-05-27_stimsim_0000_00_g0'
all_probe_folders = glob.glob(os.path.join(sess_folder, '*'))
all_probe_folders

sgl_folder, sgl_pd = sglu.sgl_file_struct(sess_folder)
sgl_folder

{'nidq': '/mnt/microdrive/birds/g4r4/Ephys/raw/20190715_02/20190715_02_undir_g0',
 'imec': {0: '/mnt/microdrive/birds/g4r4/Ephys/raw/20190715_02/20190715_02_undir_g0/20190715_02_undir_g0_imec0'}}

##### get the AP files for one imec probe

The imec file.

Here's the meaning of some of the metadata https://github.com/JaneliaSciComp/JRCLUST/wiki/.meta-file. In particular, there is an explanation of what channels in the probe are used and where they are located in the block. More detailed meta here https://github.com/billkarsh/SpikeGLX/blob/master/Markdown/Metadata.md.


In [6]:
probe_id = int(sess_par['probe'].split('_')[-1])

probe_data_folder = sgl_folder['imec'][probe_id]
probe_data_folder
ap_meta_files = glob.glob(os.path.join(probe_data_folder, '*.ap.meta'))

ap_meta_files[0]

'/mnt/microdrive/birds/g4r4/Ephys/raw/20190715_02/20190715_02_undir_g0/20190715_02_undir_g0_imec0/20190715_02_undir_g0_t0.imec0.ap.meta'

#### read a file and its meta

In [7]:
imec_meta_file_path = ap_meta_files[0]
# these should come from the .meta file
imec_meta_dict = sglu.get_imec_meta(imec_meta_file_path)

imec0 = sglu.get_imec_data(imec_meta_file_path)
n_chan = imec_meta_dict['nsavedchans'] #nSavedChans in meta file
s_f = imec0['meta']['s_f'] #30000.533148 #imSampleRate in meta file



In [8]:
s_f

30000.0

### Load the rig parameters and get the probe file, behavior trigers, etc
 - Get the rig par file
 - Get the aux channels
 - Detect onset of wav files

In [9]:
rig_par = et.get_rig_par(exp_struct)

## Scripts for sorting with Kilosort
Steps involved:
 - Make binary file with selected recs, chans
 - Set kilosort parameters
 - Make kilosort chanmap
 - Make kilosort scripts and phy parameters file (for manual curation)
 - Run the kilosort scripts (via matlab)
 - Expose the paths for manual curation
 - After curation, make the kwik file with sorted data
 - Cleanup and move metadata to permanentt locations

### prep the files with their nice formats, locations and names


In [10]:
from pipefinch.neural.sort.kilo import core as ksc

In [11]:
reload(ksc)
ks_params = {'kilo_version': 2,
             'use_gpu': 1,
            'auto_merge': 1,
            'filt_per_chan': 4,
            's_f': int(s_f),
            'n_chan': n_chan}

In [12]:
exp_struct['folders']['ksort']

'/data/experiment/microdrive/g4r4/Ephys/ksort/20190715_02/20190715_02_undir_g0'

In [13]:
reload(ksc)
reload(probes)
file_paths, out_folder = ksc.make_paths(exp_struct['folders']['ksort'])

In [14]:
file_paths

{'bin': '/data/experiment/microdrive/g4r4/Ephys/ksort/20190715_02/20190715_02_undir_g0/raw.bin',
 'params': '/data/experiment/microdrive/g4r4/Ephys/ksort/20190715_02/20190715_02_undir_g0/params.json',
 'prb': '/data/experiment/microdrive/g4r4/Ephys/ksort/20190715_02/20190715_02_undir_g0/chanMap.mat',
 'rez': '/data/experiment/microdrive/g4r4/Ephys/ksort/20190715_02/20190715_02_undir_g0/rez2.mat',
 'mat_log': '/data/experiment/microdrive/g4r4/Ephys/ksort/20190715_02/20190715_02_undir_g0/kilosort_mat.log',
 'phy_par': '/data/experiment/microdrive/g4r4/Ephys/ksort/20190715_02/20190715_02_undir_g0/params.py'}

In [15]:
reload(ksc)
reload(probes)
file_paths, out_folder = ksc.make_paths(exp_struct['folders']['ksort'])
os.makedirs(out_folder, exist_ok=True)

# make the probe file
# for now, just copy copy the probe file from defaults to dict

shutil.copyfile('/home/ezequiel/repos/Kilosort2/configFiles/chanMap_phase3b_allconnect.mat', file_paths['prb'])

# copy the binary file as the 'raw' binary file for the sorting
# this has to be done either way because /data partition is faster
# todo: concatenate them or something, from a kwd
#logger.info('copying raw file into {}'.format(file_paths['bin']))
#imec0['only_neural'].tofile(file_paths['bin'])

shutil.copyfile(sglu.get_data_meta_path(imec_meta_file_path)[0], file_paths['bin'])

# parameters to pass to the msort scripts, other than de defaults
ks_params.update({'s_f': s_f, # required,
                  'n_chan': n_chan, # total number of chans in the .bin file,
                  'dtype_name': imec0['neural'].dtype.name
            })
ksc.make_kilo_scripts(exp_struct['folders']['ksort'], ks_params)
phy_pars = ksc.make_phy_par_file(ks_params, file_paths)

2019-08-01 19:49:21,159 - pipefinch.neural.sort.kilo.core - INFO - Written kilo script /data/experiment/microdrive/g4r4/Ephys/ksort/20190715_02/20190715_02_undir_g0/master.m
2019-08-01 19:49:21,163 - pipefinch.neural.sort.kilo.core - INFO - Written kilo script /data/experiment/microdrive/g4r4/Ephys/ksort/20190715_02/20190715_02_undir_g0/config.m
2019-08-01 19:49:21,165 - pipefinch.neural.sort.kilo.core - INFO - Written kilo script /data/experiment/microdrive/g4r4/Ephys/ksort/20190715_02/20190715_02_undir_g0/run_master.m
2019-08-01 19:49:21,167 - pipefinch.neural.sort.kilo.core - INFO - Written phy parameters file /data/experiment/microdrive/g4r4/Ephys/ksort/20190715_02/20190715_02_undir_g0/params.py
2019-08-01 19:49:21,168 - pipefinch.neural.sort.kilo.core - INFO - Written ksort parameters file /data/experiment/microdrive/g4r4/Ephys/ksort/20190715_02/20190715_02_undir_g0/params.json


In [16]:
file_paths

{'bin': '/data/experiment/microdrive/g4r4/Ephys/ksort/20190715_02/20190715_02_undir_g0/raw.bin',
 'params': '/data/experiment/microdrive/g4r4/Ephys/ksort/20190715_02/20190715_02_undir_g0/params.json',
 'prb': '/data/experiment/microdrive/g4r4/Ephys/ksort/20190715_02/20190715_02_undir_g0/chanMap.mat',
 'rez': '/data/experiment/microdrive/g4r4/Ephys/ksort/20190715_02/20190715_02_undir_g0/rez2.mat',
 'mat_log': '/data/experiment/microdrive/g4r4/Ephys/ksort/20190715_02/20190715_02_undir_g0/kilosort_mat.log',
 'phy_par': '/data/experiment/microdrive/g4r4/Ephys/ksort/20190715_02/20190715_02_undir_g0/params.py'}

In [24]:
reload(ksc)
sort_result, sort_return_value = ksc.do_the_sort(file_paths)

2019-08-01 21:17:44,257 - pipefinch.neural.sort.kilo.core - INFO - Running kilosort on matlab
2019-08-01 21:17:44,258 - pipefinch.neural.sort.kilo.core - INFO - Sort folder is /data/experiment/microdrive/g4r4/Ephys/ksort/20190715_02/20190715_02_undir_g0
2019-08-01 21:17:44,259 - pipefinch.neural.sort.kilo.core - INFO - output to /data/experiment/microdrive/g4r4/Ephys/ksort/20190715_02/20190715_02_undir_g0/kilosort_mat.log


In [25]:
sort_return_value

0

In [26]:
sort_result

'MATLAB is selecting SOFTWARE OPENGL rendering.\n\n                            < M A T L A B (R) >\n                  Copyright 1984-2019 The MathWorks, Inc.\n              R2019a Update 1 (9.6.0.1099231) 64-bit (glnxa64)\n                               April 12, 2019\n\n \nTo get started, type doc.\nFor product information, visit www.mathworks.com.\n \n\n.                         config.m                  spike_clusters.npy        \n..                        kilosort_mat.log          spike_templates.npy       \n.phy                      master.m                  spike_times.npy           \namplitudes.npy            params.json               temp_wh.dat               \nchanMap.mat               params.py                 template_feature_ind.npy  \nchannel_map.npy           pc_feature_ind.npy        template_features.npy     \nchannel_positions.npy     pc_features.npy           templates.npy             \ncluster_Amplitude.tsv     phy.log                   templates_ind.npy         \ncl

In [21]:
file_paths

{'bin': '/data/experiment/microdrive/g4r4/Ephys/ksort/20190715_02/20190715_02_undir_g0/raw.bin',
 'params': '/data/experiment/microdrive/g4r4/Ephys/ksort/20190715_02/20190715_02_undir_g0/params.json',
 'prb': '/data/experiment/microdrive/g4r4/Ephys/ksort/20190715_02/20190715_02_undir_g0/chanMap.mat',
 'rez': '/data/experiment/microdrive/g4r4/Ephys/ksort/20190715_02/20190715_02_undir_g0/rez2.mat',
 'mat_log': '/data/experiment/microdrive/g4r4/Ephys/ksort/20190715_02/20190715_02_undir_g0/kilosort_mat.log',
 'phy_par': '/data/experiment/microdrive/g4r4/Ephys/ksort/20190715_02/20190715_02_undir_g0/params.py'}

## Command for viewing:
 - open up terminal with the environment msort
 - go go the ss_data folder for the session
 - run the command: phy template-gui params.py

# After manual curation
 - save the curated spikes
 - come back to the notebook and run 

In [23]:
from pipefinch.h5tools.kwik import kwikfunctions as kwkf
reload(kwkf)
reload(et)

kwkf.kilo_to_kwik(exp_struct['files']['kwd'],
                 exp_struct['files']['kwik'],
                 exp_struct['folders']['ksort'],
                 #rec_in_binary=selection_rec_list,
                 raw_format='sgl')

#sglu.all_sgl_to_kwd(sess_par, include_blocks=['adc', 'dig_in'], overwrite=True)

2019-08-01 21:17:03,752 - pipefinch.h5tools.kwik.kwikfunctions - INFO - Creating kwik file /data/experiment/microdrive/g4r4/Ephys/kwik/20190715_02/sort_1/spikes.kwik from kilosort folder /data/experiment/microdrive/g4r4/Ephys/ksort/20190715_02/20190715_02_undir_g0
2019-08-01 21:17:03,754 - pipefinch.h5tools.kwik.kwikfunctions - INFO - Found clu file, will attempt to unpack manual sorted data from kilosort
2019-08-01 21:17:03,756 - pipefinch.h5tools.kwik.kwikfunctions - INFO - 30000.0
2019-08-01 21:17:03,996 - pipefinch.h5tools.kwik.kwikfunctions - INFO - Making spike tables
2019-08-01 21:17:04,222 - pipefinch.h5tools.kwik.kwikfunctions - INFO - 30000.0
2019-08-01 21:17:04,406 - pipefinch.h5tools.kwik.kwikfunctions - INFO - Making rec tables (make_rec_groups)
2019-08-01 21:17:04,491 - pipefinch.h5tools.kwik.kwikfunctions - INFO - Making cluster group tables
2019-08-01 21:17:04,492 - pipefinch.h5tools.kwik.kwikfunctions - INFO - found cluster tags file in /data/experiment/microdrive/g4r4

IndexError: index 209 is out of bounds for axis 0 with size 209

In [73]:
exp_struct['folders']['ksort']

'/data/experiment/microdrive/g4r4/Ephys/ksort/20190715_02/20190715_02_dir_g0'

In [29]:
os.path.isfile('/data/experiment/microdrive/p14r14/Ephys/ksort/2019-02-15_3125_02/cluster_group.tsv')

True

### Generate the kwd file with the adc streams and dig inputs (for events, like preceptron)

In [36]:
# this needs to roun only once for the session. It goes trhoguh all the epochs
sglu.all_sgl_to_kwd(sess_par, include_blocks=['adc', 'dig_in'], overwrite=True)

2019-07-29 10:51:11,668 - pipefinch.pipeline.sglxutil - INFO - will process to kwd all epochs in session folder /mnt/microdrive/birds/g4r4/Ephys/raw/20190723_02
2019-07-29 10:51:11,669 - pipefinch.pipeline.sglxutil - INFO - found 2 epoch subfolders
2019-07-29 10:51:11,670 - pipefinch.pipeline.sglxutil - INFO - epoch folder /mnt/microdrive/birds/g4r4/Ephys/raw/20190723_02/raw_dir_02_1505_g0
2019-07-29 10:51:11,674 - pipefinch.pipeline.sglxutil - WARNING - File /data/experiment/microdrive/g4r4/Ephys/kwik/20190723_02/raw_dir_02_1505_g0/streams.kwd already exists
2019-07-29 10:51:11,674 - pipefinch.pipeline.sglxutil - INFO - Will create a new kwd file and overwrite the old one
2019-07-29 10:51:11,675 - pipefinch.pipeline.sglxutil - INFO - dest file: /data/experiment/microdrive/g4r4/Ephys/kwik/20190723_02/raw_dir_02_1505_g0/streams.kwd
2019-07-29 10:51:12,174 - pipefinch.pipeline.sglxutil - INFO - block adc
2019-07-29 10:51:12,556 - pipefinch.pipeline.sglxutil - INFO - block dig_in
2019-07-

# These won't work in ksort yet

In [31]:
### extract all unit waveforms
reload(kwkf)

from pipefinch.neural import units
reload(units)
units.get_all_unit_waveforms(exp_struct['files']['kwik'], exp_struct['files']['kwd'])

2019-05-21 15:42:28,359 - pipefinch.neural.units - INFO - About to get all waveforms for 3 units in file /data/experiment/microdrive/p14r14/Ephys/kwik/2019-02-15_3125_01/sort_1/spikes.kwik


0

In [ ]:
def msort_cleanup(exp_struct: dict):
    # remove the mda files and try to cleanup the msort temp location
    mda_raw_path = exp_struct['files']['mda_raw']
    logger.info('removing intermediate msort mda file {}'.format(mda_raw_path))
    os.remove(mda_raw_path)

def ksort_cleanup(exp_struct: dict):
    # remove the 
def msort_tmp_clean():
    tmp_dir = os.path.abspath(os.environ['ML_TEMPORARY_DIRECTORY'])
    logger.info('Cleaning up msort temp dir {}'.format(tmp_dir))
    
#msort_tmp_clean()
msort_cleanup(exp_struct)

In [165]:
 exp_struct['files']

{'par': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-13_1750_01/params.json',
 'set': '/mnt/zuperfinchjr/Data/p14r14/ephys/raw/2019-02-13_1750_01/settings.isf',
 'kwd': '/media/zinch/Windows/experiment/p14r14/ephys/kwik/2019-02-13_1750_01/streams.kwd',
 'kwik': '/media/zinch/Windows/experiment/p14r14/ephys/kwik/2019-02-13_1750_01/spikes.kwik',
 'kwe': '/media/zinch/Windows/experiment/p14r14/ephys/kwik/2019-02-13_1750_01/events.kwe',
 'mda_raw': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-13_1750_01/raw.mda'}

## Dig into Ksort output files
Looking for:
    - main channels
    - spike waveforms

In [51]:
templates_path = os.path.join(exp_struct['folders']['ksort'], 'pc_features.npy')

pc = np.load(os.path.join(exp_struct['folders']['ksort'], 
                          'pc_features.npy'))

pc_ind = np.load(os.path.join(exp_struct['folders']['ksort'], 
                          'pc_feature_ind.npy'))

In [52]:
pc_ind.shape

(196, 32)

In [72]:
pc_ind[1]

array([ 1,  3,  0,  2,  5,  4,  7,  6,  9,  8, 11, 10, 13, 12, 15, 14, 17,
       16, 19, 18, 21, 20, 23, 22, 25, 24, 27, 26, 29, 28, 31, 30],
      dtype=uint32)

In [53]:
pc.shape

(2475599, 3, 32)

In [67]:
xx = np.load(os.path.join(exp_struct['folders']['ksort'], 
                          'templates.npy'))
xx.shape

(196, 82, 362)